In [1]:
from elasticsearch import Elasticsearch 

es=Elasticsearch([{'host':'52.188.166.61','port':9200}])


In [2]:
e1={
    "first_name":"nitin",
    "last_name":"panwar",
    "age": 27,
    "about": "Love to play cricket",
    "interests": ['sports','music'],
}

# Inserting a document
res = es.index(index='megacorp',doc_type='employee',id=1,body=e1)


In [3]:
print(res)

{'_index': 'megacorp', '_type': 'employee', '_id': '1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}


In [14]:
# Let's insert some more documents
e2={
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}
e3={
    "first_name" :  "Douglas",
    "last_name" :   "Fir",
    "age" :         35,
    "about":        "I like to build cabinets",
    "interests":  [ "forestry" ]
}
e4={
    "first_name":"asd",
    "last_name":"pafdfd",
    "age": 27,
    "about": "Love to play football",
    "interests": ['sports','music'],
}
res=es.index(index='megacorp',doc_type='employee',id=2,body=e2)
print(res)
res=es.index(index='megacorp',doc_type='employee',id=3,body=e3)
print(res)
res=es.index(index='megacorp',doc_type='employee',id=4,body=e4)
print(res)

{'_index': 'megacorp', '_type': 'employee', '_id': '2', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6, '_primary_term': 1}
{'_index': 'megacorp', '_type': 'employee', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7, '_primary_term': 1}
{'_index': 'megacorp', '_type': 'employee', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 8, '_primary_term': 1}


In [6]:
# Retrieving a Document:
res=es.get(index='megacorp',doc_type='employee',id=3)
print(res)

{'_index': 'megacorp', '_type': 'employee', '_id': '3', '_version': 1, '_seq_no': 4, '_primary_term': 1, 'found': True, '_source': {'first_name': 'Douglas', 'last_name': 'Fir', 'age': 35, 'about': 'I like to build cabinets', 'interests': ['forestry']}}


In [7]:
# Deleting a document:
res=es.delete(index='megacorp',doc_type='employee',id=3)
print

{'_index': 'megacorp', '_type': 'employee', '_id': '3', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 1}


In [9]:
res= es.search(index='megacorp',body={'query':{'match_all':{}}})
print(res['hits']['total'])

{'value': 2, 'relation': 'eq'}


In [12]:
# Search Lite:
# bool operator:
# bool takes a dictionary containing at least one of must, should, and must_not, each of which takes a list of matches or other further search operators.

res= es.search(index='megacorp',body={
        'query':{
            'bool':{
                'must':[{
                        'match':{
                            'first_name':'nitin'
                        }
                    }]
            }
        }
    })
print(res['hits']['hits'])

[{'_index': 'megacorp', '_type': 'employee', '_id': '1', '_score': 1.2039728, '_source': {'first_name': 'nitin', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}}]


In [13]:
# Filter Operator 
# only employees who are older than 25. Our query will change a little to accommodate a filter
res= es.search(index='megacorp',body={
        'query':{
            'bool':{
                'must':{
                    'match':{
                        'first_name':'nitin'
                    }
                },
                "filter":{
                    "range":{
                        "age":{
                            "gt":25
                        }
                    }
                }
            }
        }
    })

print(res['hits']['hits'])

[{'_index': 'megacorp', '_type': 'employee', '_id': '1', '_score': 1.2039728, '_source': {'first_name': 'nitin', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}}]


In [23]:
# Full text search
res= es.search(index='megacorp',doc_type='employee',body={
        'query':{
            'match':{
                "about":"play cricket"
            }
        }
    })
for hit in res['hits']['hits']:
    print(hit['_source']['about']) 
    print(hit['_score'])
    print('**********************')

Love to play cricket
2.0281231
**********************
Love to play football
0.74101156
**********************


In [22]:
# Phrase Search
# Finding individual words in a field is all well and good, but sometimes you want to match exact sequence of words of phrases.
res= es.search(index='megacorp',doc_type='employee',body={
        'query':{
            'match_phrase':{
                "about":"play cricket"
            }
        }
    })
for hit in res['hits']['hits']:
    print(hit['_source']['about'])
    print(hit['_score'])
    print('**********************')

Love to play cricket
2.0281231
**********************


In [27]:
# autocomplete suggestion
res = es.search(index="megacorp", 
body={
  "suggest": {
    "field-suggest": {
      "prefix": "play",
      "completion": {
        "field": "about",
        "fuzzy": {
          "fuzziness": 2
        }
      }
    }
  }
})

print(res)

RequestError: RequestError(400, 'search_phase_execution_exception', 'Field [about] is not a completion suggest field')